## Svm classifier


In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import TimeSeriesSplit
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_validate
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn import preprocessing

In [2]:
dataset = pd.read_csv('../Data/imputedWQ.csv', parse_dates=True, header=0, index_col=0)

In [4]:
testset = pd.read_csv('../Data/testing2018-mean_imputed.csv', parse_dates=True, header=0, index_col=0)

In [5]:
cols = list(dataset.columns.values)
cols = cols[:-1]
dataset[cols] = dataset[cols].rolling(window=10).mean()
dataset = dataset.iloc[10:]

In [6]:
X = dataset.drop('EVENT', axis=1)  
y = dataset['EVENT'] 

In [32]:
# print(classification_report(y_test,y_pred)) 

# TimeSeriesSplit
## Special case of KFold
### Param: n_splits according to the number of months

P.s: even though this probably would not result in accurate month separation, probably would go more with accurate samples separation

In [7]:
tss = TimeSeriesSplit(n_splits=4)
# kn = KNeighborsClassifier(n_neighbors=5, algorithm='brute') 
# sc = cross_val_score(kn, X, y, cv=tss, scoring='accuracy')

In [7]:
dataset.groupby(dataset.index.month).sum()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
8,290697.052632,6799.713362,343434.304450,3.095388e+07,8.662770e+06,786979.749453,4.008524e+06,6.737174e+07,3.885911e+07,165.0
9,353955.600000,7172.806251,361367.922324,3.252037e+07,9.111778e+06,865139.886451,4.396339e+06,6.858679e+07,4.056066e+07,1310.0
10,431429.980435,7384.744348,374076.598978,3.364092e+07,9.410066e+06,710265.240783,4.398468e+06,6.299556e+07,4.047151e+07,174.0
11,112343.550000,1736.255000,88672.950000,7.956982e+06,2.056477e+06,144182.960000,9.913322e+05,1.533778e+07,9.644876e+06,77.0


In [8]:
testset.groupby(testset.index.month).sum()

,Tp,Cl,pH,Redox,Leit,Trueb,Cl_2,Fm,Fm_2,EVENT
Time,,,,,,,,,,
1,168946.9,6486.070000,367918.485157,3.433636e+07,1.030073e+07,810.50142,4804.052413,65217445.0,45054296.0,773.0
2,65273.9,2560.330000,144423.580000,1.363104e+07,3.474466e+06,286.97000,1847.039119,26277783.0,18979849.0,895.0
11,183728.9,3401.380063,177387.328683,1.620504e+07,4.328124e+06,425.34816,2341.372000,30974768.0,19343072.0,352.0
12,171830.7,4713.325094,260945.894188,2.417727e+07,7.182003e+06,634.66800,3317.529000,44719245.0,26572835.0,309.0


## TimeSeries CV

- Applying TimeSeriesSplit in cross validation with multiple scores
    - F1-macro (alter. f1_weighted, f1_micro)
    - Precision
    - Recall
    
P.s errors result because the classifier on specific folds couldn't identify at all one class and probably gave just true or false ( I guess false in this case because of high class-imbalance)

In [8]:
scoring = ['f1', 'f1_micro', 'f1_macro', 'precision', 'recall']

In [9]:
scaler = preprocessing.StandardScaler()
X_train_standard = scaler.fit_transform(X)

svm_clf = SVC(C=100, gamma=0.1 )           
              
results = cross_validate(svm_clf, X, y, cv=tss, scoring=scoring, n_jobs=-1)

/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/envs/py3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/syka/anaconda2/e

In [12]:
print("Model(svm, c=100, gamma=0.1): ({0}, {1}, {2}, {3},{4})\n".format(results['test_f1'].mean(), results['test_f1_micro'].mean(), results['test_f1_macro'].mean(), results['test_precision'].mean(), results['test_recall'].mean()))

Model(svm, c=100, gamma=0.1): (0.0, 0.98582995951417, 0.4964091429827894, 0.0,0.0)



In [12]:
svm_clf.fit(X_train_standard, y)

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [13]:
y_pred = svm_clf.predict(scaler.transform(X))

In [14]:
f1_score(y_pred, y)
#f1_score(y_pred, y, average='macro')

0.7041144901610018

## GridSearchCV
### Hyperparameter optimizations for RandForest classifier

- Scoring F1_macro can be changed to precision, recall or f1_weighted or f1_micro and possibly you would get different parameters.

In [15]:
#from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import GridSearchCV

param_grid = [{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf'],'class_weight': ['balanced']},
 ]  


grid = GridSearchCV(SVC(), param_grid, cv=tss, scoring="f1_macro", n_jobs=-1)

In [16]:
grid.fit(X,y)

GridSearchCV(cv=TimeSeriesSplit(max_train_size=None, n_splits=4),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid=[{'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf'], 'class_weight': ['balanced']}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1_macro', verbose=0)

In [17]:
grid.best_params_

{'C': 1000, 'class_weight': 'balanced', 'gamma': 0.0001, 'kernel': 'rbf'}

In [18]:
model = grid.best_estimator_

In [19]:
results = cross_validate(model, X, y, cv=tss, scoring=scoring, n_jobs=-1)

In [20]:
print("Model(svm, c:1000,balanced, gamma=0.0001, 'rbf'): ({0}, {1}, {2})\n".format(results['test_f1'].mean(), results['test_f1_macro'].mean(), results['test_precision'].mean(), results['test_recall'].mean()))

Model(svm, c:1000,balanced, gamma=0.0001, 'rbf'): (0.09522279844434825, 0.5292396679470368, 0.12475791994255342)



# Testing on Test set

In [10]:
cols = list(testset.columns.values)
cols = cols[:-1]
testset[cols] = testset[cols].rolling(window=10).mean()
testset = testset.iloc[10:]

In [11]:
X_test = testset.drop('EVENT', axis=1)  
y_test = testset['EVENT']  

In [23]:
predicted = model.predict(X_test)

In [24]:
predicted = predicted<0.5

In [25]:
f1_score(y_test,predicted, average='macro')

0.11304139079275452